In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk import word_tokenize as wt
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BVILLAMIL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('Twitter.csv')
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1
...,...,...
162964,why these 456 crores paid neerav modi not reco...,-1
162965,dear rss terrorist payal gawar what about modi...,-1
162966,did you cover her interaction forum where she ...,0
162967,there big project came into india modi dream p...,0


In [3]:
df = df.dropna()
df = df.reset_index(drop=True)
df['category'].value_counts()

 1    72249
 0    55211
-1    35509
Name: category, dtype: int64

In [4]:
df = df.sample(frac=1).reset_index(drop=True)
data = df[df['category']==1][:35000]
data = data.append(df[df['category']==-1][:35000])
data = data.append(df[df['category']==0][:35000])
data = data.reset_index(drop=True)
display(data['category'].value_counts())

C:\Users\BVILLAMIL\AppData\Local\Temp\ipykernel_18784\2934771828.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df[df['category']==-1][:35000])
C:\Users\BVILLAMIL\AppData\Local\Temp\ipykernel_18784\2934771828.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df[df['category']==0][:35000])


 1    35000
-1    35000
 0    35000
Name: category, dtype: int64

In [5]:
data['pre_process'] = data['clean_text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['pre_process'] = data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
data['pre_process'] = data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

c:\Users\BVILLAMIL\Desktop\revalida2\env\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
def contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"wouldn't", "would not", s)
    s = re.sub(r"couldn't", "could not", s)
    s = re.sub(r"\'d", "would", s)
    s = re.sub(r"can\'t", "can not", s)
    s = re.sub(r"n\'t", "not", s)
    s = re.sub(r"\'re", "are", s)
    s = re.sub(r"\'s", "is", s)
    s = re.sub(r"\'ll", "will", s)
    s = re.sub(r"\'t", "not", s)
    s = re.sub(r"\'ve", "have", s)
    s = re.sub(r"\'m", "am", s)
    return s

data['pre_process'] = data['pre_process'].apply(lambda x:contractions(x))

In [7]:
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in wt(x)]))

In [8]:
stop = stopwords.words('english')
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in wt(x)]))

In [9]:
data

,clean_text,category,pre_process
0,its good that only raid happened not like you ...,1,good raid happened like made allegation cbi wa...
1,yes not bhakt congress was different then talk...,1,yes bhakt congress different talking sonia era...
2,banks sold 1000 crores worth united breweries ...,1,bank sold crore worth united brewery mcdowell ...
3,constantly bashing modi will counterproductive...,1,constantly bashing modi counterproductive popu...
4,modi was unethical when announce the scheme th...,1,modi unethical announce scheme wayhe even star...
...,...,...,...
104995,nationalist will vote for modi else prove your...,0,nationalist vote modi else prove porki anti na...
104996,why dont you also try against modi,0,dont also try modi
104997,exchief mission shakti upa didn’ give nod but ...,0,exchief mission shakti upa give nod courage vi...
104998,will ensure that china international communiti...,0,ensure china international community push spac...


In [10]:
x = data['pre_process']
y = data['category']

vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_vectorized, y, test_size=0.25, random_state=30)
print("Train:", x_train.shape, y_train.shape)
print("Test: ", x_test.shape, y_test.shape)

Train: (78750, 73429) (78750,)
Test:  (26250, 73429) (26250,)


In [12]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(x_train, y_train)

In [13]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(x_train, y_train)
model = gs_clf.best_estimator_

In [16]:
y_pred = model.predict(x_test)

report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.88      0.84      0.86      8772
           0       0.87      0.94      0.90      8760
           1       0.89      0.85      0.87      8718

    accuracy                           0.88     26250
   macro avg       0.88      0.88      0.88     26250
weighted avg       0.88      0.88      0.88     26250



Testing the Sentiment Analysis
-1 = Negative
 0 = Neutral
 1 = Positive

In [27]:
test_feature = vectorizer.transform(["this sucks"])
model.predict(test_feature)

array([0], dtype=int64)

In [ ]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models'+".p", "wb"))